In [2]:
import os
import tensorflow as tf
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, mean_absolute_error
from scipy.stats import bootstrap
import numpy as np 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import numpy as np
import pandas as pd

# ============================= T2D-dementia Fluctuating_Decreasing cluster =============================

# =============================== 配置区 ===============================
DATA_PATH = "E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Fluctuating_Decreasing.csv" 
RANDOM_STATE = 42
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-5 

# ============================= 1.数据准备 =============================
print("\n1. 数据加载与预处理...")

# 加载数据
df = pd.read_csv("E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Fluctuating_Decreasing.csv")
protein_names = df.columns[9:76].tolist()
covariates_names = df.columns[2:9].tolist()
num_proteins = len(protein_names)

# 数据拆分
y = df.iloc[:, 1].values.astype(np.int32)
X_protein = df.iloc[:, 9:76].values.astype(np.float32)
X_covariates = df.iloc[:, 2:9].values.astype(np.float32)

# 缺失值处理
imputer = SimpleImputer(strategy='mean')
X_protein = imputer.fit_transform(X_protein)
X_covariates = imputer.fit_transform(X_covariates)

# 标准化
scaler = StandardScaler()
X_protein_scaled = scaler.fit_transform(X_protein)
X_covariates_scaled = scaler.fit_transform(X_covariates)
X_combined = np.hstack([X_protein_scaled, X_covariates_scaled])

# 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=RANDOM_STATE
)

# SMOTE过采样
sm = SMOTE(random_state=RANDOM_STATE)
X_train, y_train = sm.fit_resample(X_train, y_train)

# Further split training data into sub-train and validation for grid search
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=RANDOM_STATE
)

# Define grid search parameters
layer_configs = [[1024, 512, 256, 128], [512, 256, 128], [256, 128]]  # Varying complexity
dropout_rates = [0.2, 0.3, 0.4]
l2_lambdas = [0.001, 0.01, 0.1]
learning_rates = [1e-5, 5e-5, 1e-4]

# Function to build model with variable hyperparameters
class MonotonicConstraint(tf.keras.constraints.Constraint):
    """强制阈值参数单调递增约束"""
    def __call__(self, w):
        return tf.cumsum(tf.nn.elu(w) + 1e-6)

def build_model(input_dim, num_classes, layer_sizes, dropout_rate, l2_lambda, learning_rate):
    class DeepOrdinal(tf.keras.Model):
        def __init__(self, input_dim, num_classes):
            super().__init__()
            self.num_classes = num_classes
            
            # 网络结构
            self.dense_stack = tf.keras.Sequential()
            for size in layer_sizes:
                self.dense_stack.add(tf.keras.layers.Dense(size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
                self.dense_stack.add(tf.keras.layers.Dropout(dropout_rate))
            self.output_layer = tf.keras.layers.Dense(num_classes-1, kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))
            
            # 可训练阈值参数
            self.thresholds = tf.Variable(
                initial_value=tf.sort(tf.linspace(-1.0, 1.0, num_classes-1)),
                trainable=True,
                constraint=MonotonicConstraint(),
                name="ordinal_thresholds"
            )

        def call(self, inputs):
            x = self.dense_stack(inputs)
            return self.output_layer(x)

        def custom_loss(self, y_true, y_pred):
            """序数回归损失函数"""
            y_true = tf.cast(tf.reshape(y_true, (-1, 1)), tf.float32)
            cum_loss = 0.0
            
            for k in range(self.num_classes-1):
                target = tf.cast(y_true > k, tf.float32)
                logit = y_pred[:, k] - self.thresholds[k]
                
                # 添加数值稳定化
                logit = tf.clip_by_value(logit, -10.0, 10.0)
                logit = tf.reshape(logit, (-1, 1))
                
                loss = tf.nn.sigmoid_cross_entropy_with_logits(target, logit)
                cum_loss += tf.reduce_mean(loss)
                
            return cum_loss

    model = DeepOrdinal(input_dim=input_dim, num_classes=num_classes)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        clipvalue=1.0  # 梯度裁剪
    )
    model.compile(
        optimizer=optimizer,
        loss=model.custom_loss,
        metrics=['accuracy']
    )
    return model

# Grid search loop
best_score = float('inf')  # Minimize validation MAE
best_params = None
results = []  # Track all configurations

for layer_sizes, dropout_rate, l2_lambda, lr in itertools.product(layer_configs, dropout_rates, l2_lambdas, learning_rates):
    print(f"Evaluating config: layers={layer_sizes}, dropout={dropout_rate}, l2={l2_lambda}, lr={lr}")
    
    model = build_model(X_subtrain.shape[1], num_classes=5, layer_sizes=layer_sizes, 
                        dropout_rate=dropout_rate, l2_lambda=l2_lambda, learning_rate=lr)
    
    # Train with class weights
    history = model.fit(
        X_subtrain, y_subtrain,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        class_weight=class_weight,
        verbose=0
    )
    
    # Predict on validation and compute MAE
    y_val_pred = ordinal_predict(model, X_val)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    results.append({
        'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr, 'val_mae': val_mae
    })
    
    if val_mae < best_score:
        best_score = val_mae
        best_params = {'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr}

# Save results to CSV for review
pd.DataFrame(results).to_csv('E:/neuro_od/T2D-痴呆结果/grid_search_results.csv', index=False)

print(f"Best configuration: {best_params} with validation MAE: {best_score}")

# Now, use best_params to build and train the final model on full X_train, y_train
model = build_model(
    input_dim=X_train.shape[1],
    num_classes=5,
    layer_sizes=best_params['layers'],
    dropout_rate=best_params['dropout'],
    l2_lambda=best_params['l2'],
    learning_rate=best_params['lr']
)

# ============================ 3.模型训练 =============================
print("\n3. 模型训练中...")

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    class_weight=class_weight,
    verbose=2
)


# =========================== 4.模型评估 =============================
print("\n4. 模型评估...")

y_pred = ordinal_predict(model, X_test)
print(classification_report(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


1. 数据加载与预处理...


C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 

In [3]:
# ===================== 5.梯度重要性计算（蛋白质部分）=====================
print("\n5. 计算梯度重要性...")

@tf.function
def compute_gradients(inputs):
    with tf.GradientTape() as tape:
        tape.watch(inputs)  # 显式监视输入
        preds = model(inputs)
    return tape.gradient(preds, inputs)

# 计算所有训练样本的梯度均值
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
gradients = compute_gradients(X_train_tensor)

if gradients is None:
    raise ValueError("梯度计算失败，请检查模型输入输出依赖关系")

# 调整聚合维度（假设输入为二维张量）
abs_gradients = tf.abs(gradients).numpy()  # Convert to NumPy for bootstrapping
protein_abs_gradients = abs_gradients[:, :num_proteins]  # Extract protein part (first num_proteins columns)

# Bootstrapping function for mean importance per protein
def bootstrap_mean(data):
    return np.mean(data, axis=0)  # Mean across samples for each protein

# Perform bootstrapping (95% CI, 1000 resamples)
boot_result = bootstrap((protein_abs_gradients,), bootstrap_mean, n_resamples=1000, random_state=RANDOM_STATE, method='percentile')

# Extract statistics
gradient_importance_mean = boot_result.bootstrap_distribution.mean(axis=-1)  # Mean of bootstraps across resamples; results in shape (67,)
gradient_importance_se = boot_result.standard_error
gradient_importance_ci_low = boot_result.confidence_interval.low
gradient_importance_ci_high = boot_result.confidence_interval.high

# Diagnostic prints to verify lengths (optional; can be removed after confirmation)
print("Length of protein_names:", len(protein_names))
print("Shape of gradient_importance_mean:", gradient_importance_mean.shape)
print("Shape of gradient_importance_se:", gradient_importance_se.shape)
print("Shape of gradient_importance_ci_low:", gradient_importance_ci_low.shape)
print("Shape of gradient_importance_ci_high:", gradient_importance_ci_high.shape)

# Save results to CSV
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': gradient_importance_mean,
    'Gradient Importance SE': gradient_importance_se,
    'Gradient Importance CI Lower (95%)': gradient_importance_ci_low,
    'Gradient Importance CI Upper (95%)': gradient_importance_ci_high
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_dementia_Fluctuating_Decreasing_深度有序回归_with_ci.csv', index=False)


5. 计算梯度重要性...
Length of protein_names: 67
Shape of gradient_importance_mean: (67,)
Shape of gradient_importance_se: (67,)
Shape of gradient_importance_ci_low: (67,)
Shape of gradient_importance_ci_high: (67,)


In [ ]:
from scipy.stats import percentileofscore

# Permutation Test Parameters
n_permutations = 1000  # Adjust for computational feasibility

# Function to compute mean absolute gradients
def compute_mean_abs_gradients(inputs):
    grads = compute_gradients(tf.convert_to_tensor(inputs, dtype=tf.float32))
    return np.mean(np.abs(grads.numpy()), axis=0)[:num_proteins]  # Protein part only

# Original mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Generate null distribution
null_distribution = np.zeros((n_permutations, num_proteins))
for i in range(n_permutations):
    # Permute features (columns) independently for each protein
    permuted_X = X_train.copy()
    for j in range(num_proteins):
        np.random.shuffle(permuted_X[:, j])
    null_distribution[i] = compute_mean_abs_gradients(permuted_X)

# Compute p-values (one-tailed: proportion of null means >= original)
p_values_perm = np.array([1 - (percentileofscore(null_distribution[:, j], original_mean[j]) / 100) for j in range(num_proteins)])

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': original_mean,
    'P-value (Permutation)': p_values_perm
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_permutation_test_Fluctuating_Decreasing.csv', index=False)

In [ ]:
# Simplified; insert after model training
from scipy.stats import norm

# Compute mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Approximate pivotal statistic (e.g., standardized gradient mean)
std_gradients = np.std(protein_abs_gradients, axis=0)
pivotal_stats = original_mean / (std_gradients / np.sqrt(protein_abs_gradients.shape[0]))
p_values_pivotal = 2 * (1 - norm.cdf(np.abs(pivotal_stats)))  # Two-tailed

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Pivotal Statistic': pivotal_stats,
    'P-value (Pivotal)': p_values_pivotal
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_pivotal_test_Fluctuating_Decreasing.csv', index=False)

In [4]:

# ============================= T2D-痴呆结果 Fluctuating_Increasing =============================

# =============================== 配置区 ===============================
DATA_PATH = "E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Fluctuating_Increasing.csv"  # 修改为实际路径
RANDOM_STATE = 42
BATCH_SIZE = 64
EPOCHS = 30
SHAP_SAMPLE_SIZE = 500  # SHAP计算样本量
LEARNING_RATE = 1e-5  # 调低学习率

# ============================= 1.数据准备 =============================
print("\n1. 数据加载与预处理...")

# 加载数据
df = pd.read_csv("E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Fluctuating_Increasing.csv")
protein_names = df.columns[9:485].tolist()
covariates_names = df.columns[2:9].tolist()
num_proteins = len(protein_names)

# 数据拆分
y = df.iloc[:, 1].values.astype(np.int32)
X_protein = df.iloc[:, 9:485].values.astype(np.float32)
X_covariates = df.iloc[:, 2:9].values.astype(np.float32)

# 缺失值处理
imputer = SimpleImputer(strategy='mean')
X_protein = imputer.fit_transform(X_protein)
X_covariates = imputer.fit_transform(X_covariates)

# 标准化
scaler = StandardScaler()
X_protein_scaled = scaler.fit_transform(X_protein)
X_covariates_scaled = scaler.fit_transform(X_covariates)
X_combined = np.hstack([X_protein_scaled, X_covariates_scaled])

# 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=RANDOM_STATE
)

# SMOTE过采样
sm = SMOTE(random_state=RANDOM_STATE)
X_train, y_train = sm.fit_resample(X_train, y_train)

# Further split training data into sub-train and validation for grid search
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=RANDOM_STATE
)

# Define grid search parameters
layer_configs = [[1024, 512, 256, 128], [512, 256, 128], [256, 128]]  # Varying complexity
dropout_rates = [0.2, 0.3, 0.4]
l2_lambdas = [0.001, 0.01, 0.1]
learning_rates = [1e-5, 5e-5, 1e-4]

# Function to build model with variable hyperparameters
class MonotonicConstraint(tf.keras.constraints.Constraint):
    """强制阈值参数单调递增约束"""
    def __call__(self, w):
        return tf.cumsum(tf.nn.elu(w) + 1e-6)

def build_model(input_dim, num_classes, layer_sizes, dropout_rate, l2_lambda, learning_rate):
    class DeepOrdinal(tf.keras.Model):
        def __init__(self, input_dim, num_classes):
            super().__init__()
            self.num_classes = num_classes
            
            # 网络结构
            self.dense_stack = tf.keras.Sequential()
            for size in layer_sizes:
                self.dense_stack.add(tf.keras.layers.Dense(size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
                self.dense_stack.add(tf.keras.layers.Dropout(dropout_rate))
            self.output_layer = tf.keras.layers.Dense(num_classes-1, kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))
            
            # 可训练阈值参数
            self.thresholds = tf.Variable(
                initial_value=tf.sort(tf.linspace(-1.0, 1.0, num_classes-1)),
                trainable=True,
                constraint=MonotonicConstraint(),
                name="ordinal_thresholds"
            )

        def call(self, inputs):
            x = self.dense_stack(inputs)
            return self.output_layer(x)

        def custom_loss(self, y_true, y_pred):
            """序数回归损失函数"""
            y_true = tf.cast(tf.reshape(y_true, (-1, 1)), tf.float32)
            cum_loss = 0.0
            
            for k in range(self.num_classes-1):
                target = tf.cast(y_true > k, tf.float32)
                logit = y_pred[:, k] - self.thresholds[k]
                
                # 添加数值稳定化
                logit = tf.clip_by_value(logit, -10.0, 10.0)
                logit = tf.reshape(logit, (-1, 1))
                
                loss = tf.nn.sigmoid_cross_entropy_with_logits(target, logit)
                cum_loss += tf.reduce_mean(loss)
                
            return cum_loss

    model = DeepOrdinal(input_dim=input_dim, num_classes=num_classes)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        clipvalue=1.0  # 梯度裁剪
    )
    model.compile(
        optimizer=optimizer,
        loss=model.custom_loss,
        metrics=['accuracy']
    )
    return model

# Grid search loop
best_score = float('inf')  # Minimize validation MAE
best_params = None
results = []  # Track all configurations

for layer_sizes, dropout_rate, l2_lambda, lr in itertools.product(layer_configs, dropout_rates, l2_lambdas, learning_rates):
    print(f"Evaluating config: layers={layer_sizes}, dropout={dropout_rate}, l2={l2_lambda}, lr={lr}")
    
    model = build_model(X_subtrain.shape[1], num_classes=5, layer_sizes=layer_sizes, 
                        dropout_rate=dropout_rate, l2_lambda=l2_lambda, learning_rate=lr)
    
    # Train with class weights
    history = model.fit(
        X_subtrain, y_subtrain,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        class_weight=class_weight,
        verbose=0
    )
    
    # Predict on validation and compute MAE
    y_val_pred = ordinal_predict(model, X_val)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    results.append({
        'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr, 'val_mae': val_mae
    })
    
    if val_mae < best_score:
        best_score = val_mae
        best_params = {'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr}

# Save results to CSV for review
pd.DataFrame(results).to_csv('E:/neuro_od/T2D-痴呆结果/grid_search_results.csv', index=False)

print(f"Best configuration: {best_params} with validation MAE: {best_score}")

# Now, use best_params to build and train the final model on full X_train, y_train
model = build_model(
    input_dim=X_train.shape[1],
    num_classes=5,
    layer_sizes=best_params['layers'],
    dropout_rate=best_params['dropout'],
    l2_lambda=best_params['l2'],
    learning_rate=best_params['lr']
)

# ============================ 3.模型训练 =============================
print("\n3. 模型训练中...")

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    class_weight=class_weight,
    verbose=2
)


# =========================== 4.模型评估 =============================
print("\n4. 模型评估...")

y_pred = ordinal_predict(model, X_test)
print(classification_report(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


1. 数据加载与预处理...


C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 

C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [ ]:
# ===================== 5.梯度重要性计算（蛋白质部分）=====================
print("\n5. 计算梯度重要性...")

@tf.function
def compute_gradients(inputs):
    with tf.GradientTape() as tape:
        tape.watch(inputs)  # 显式监视输入
        preds = model(inputs)
    return tape.gradient(preds, inputs)

# 计算所有训练样本的梯度均值
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
gradients = compute_gradients(X_train_tensor)

if gradients is None:
    raise ValueError("梯度计算失败，请检查模型输入输出依赖关系")

# 调整聚合维度（假设输入为二维张量）
abs_gradients = tf.abs(gradients).numpy()  # Convert to NumPy for bootstrapping
protein_abs_gradients = abs_gradients[:, :num_proteins]  # Extract protein part (first num_proteins columns)

# Bootstrapping function for mean importance per protein
def bootstrap_mean(data):
    return np.mean(data, axis=0)  # Mean across samples for each protein

# Perform bootstrapping (95% CI, 1000 resamples)
boot_result = bootstrap((protein_abs_gradients,), bootstrap_mean, n_resamples=1000, random_state=RANDOM_STATE, method='percentile')

# Extract statistics
gradient_importance_mean = boot_result.bootstrap_distribution.mean(axis=-1)  # Mean of bootstraps across resamples; results in shape (67,)
gradient_importance_se = boot_result.standard_error
gradient_importance_ci_low = boot_result.confidence_interval.low
gradient_importance_ci_high = boot_result.confidence_interval.high

# Diagnostic prints to verify lengths (optional; can be removed after confirmation)
print("Length of protein_names:", len(protein_names))
print("Shape of gradient_importance_mean:", gradient_importance_mean.shape)
print("Shape of gradient_importance_se:", gradient_importance_se.shape)
print("Shape of gradient_importance_ci_low:", gradient_importance_ci_low.shape)
print("Shape of gradient_importance_ci_high:", gradient_importance_ci_high.shape)

# Save results to CSV
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': gradient_importance_mean,
    'Gradient Importance SE': gradient_importance_se,
    'Gradient Importance CI Lower (95%)': gradient_importance_ci_low,
    'Gradient Importance CI Upper (95%)': gradient_importance_ci_high
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_dementia_Fluctuating_Increasing_深度有序回归_with_ci.csv', index=False)

In [ ]:
from scipy.stats import percentileofscore

# Permutation Test Parameters
n_permutations = 1000  # Adjust for computational feasibility

# Function to compute mean absolute gradients
def compute_mean_abs_gradients(inputs):
    grads = compute_gradients(tf.convert_to_tensor(inputs, dtype=tf.float32))
    return np.mean(np.abs(grads.numpy()), axis=0)[:num_proteins]  # Protein part only

# Original mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Generate null distribution
null_distribution = np.zeros((n_permutations, num_proteins))
for i in range(n_permutations):
    # Permute features (columns) independently for each protein
    permuted_X = X_train.copy()
    for j in range(num_proteins):
        np.random.shuffle(permuted_X[:, j])
    null_distribution[i] = compute_mean_abs_gradients(permuted_X)

# Compute p-values (one-tailed: proportion of null means >= original)
p_values_perm = np.array([1 - (percentileofscore(null_distribution[:, j], original_mean[j]) / 100) for j in range(num_proteins)])

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': original_mean,
    'P-value (Permutation)': p_values_perm
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_permutation_test_Fluctuating_increasing.csv', index=False)

In [ ]:
# 5. Pivotal Test
original_mean = np.mean(protein_abs_gradients, axis=0)
std_gradients = np.std(protein_abs_gradients, axis=0)
epsilon = 1e-10
std_gradients = np.maximum(std_gradients, epsilon)
pivotal_stats = original_mean / (std_gradients / np.sqrt(protein_abs_gradients.shape[0]))
p_values_pivotal = 2 * (1 - norm.cdf(np.abs(pivotal_stats)))

pd.DataFrame({
    'Protein': protein_names,
    'Pivotal Statistic': pivotal_stats,
    'P-value (Pivotal)': p_values_pivotal
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_pivotal_test_Fluctuating_increasing.csv', index=False)

In [6]:
import os
import tensorflow as tf
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, mean_absolute_error
from scipy.stats import bootstrap
import itertools

# ============================= T2D-痴呆结果 Gradually_Increasing =============================

# =============================== 配置区 ===============================
DATA_PATH = "E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Gradually_Increasing.csv"  # 修改为实际路径
RANDOM_STATE = 42
BATCH_SIZE = 64
EPOCHS = 30
SHAP_SAMPLE_SIZE = 500  # SHAP计算样本量
LEARNING_RATE = 1e-5  # 调低学习率

# ============================= 1.数据准备 =============================
print("\n1. 数据加载与预处理...")

# 加载数据
df = pd.read_csv(DATA_PATH)
protein_names = df.columns[9:417].tolist()
covariates_names = df.columns[2:9].tolist()
num_proteins = len(protein_names)

# 数据拆分
y = df.iloc[:, 1].values.astype(np.int32)
X_protein = df.iloc[:, 9:417].values.astype(np.float32)
X_covariates = df.iloc[:, 2:9].values.astype(np.float32)

# 缺失值处理
imputer = SimpleImputer(strategy='mean')
X_protein = imputer.fit_transform(X_protein)
X_covariates = imputer.fit_transform(X_covariates)

# 标准化
scaler = StandardScaler()
X_protein_scaled = scaler.fit_transform(X_protein)
X_covariates_scaled = scaler.fit_transform(X_covariates)
X_combined = np.hstack([X_protein_scaled, X_covariates_scaled])

# 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=RANDOM_STATE
)

# SMOTE过采样
sm = SMOTE(random_state=RANDOM_STATE)
X_train, y_train = sm.fit_resample(X_train, y_train)

# Further split training data into sub-train and validation for grid search
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=RANDOM_STATE
)

# Define grid search parameters
layer_configs = [[1024, 512, 256, 128], [512, 256, 128], [256, 128]]  # Varying complexity
dropout_rates = [0.2, 0.3, 0.4]
l2_lambdas = [0.001, 0.01, 0.1]
learning_rates = [1e-5, 5e-5, 1e-4]

# Function to build model with variable hyperparameters
class MonotonicConstraint(tf.keras.constraints.Constraint):
    """强制阈值参数单调递增约束"""
    def __call__(self, w):
        return tf.cumsum(tf.nn.elu(w) + 1e-6)

def build_model(input_dim, num_classes, layer_sizes, dropout_rate, l2_lambda, learning_rate):
    class DeepOrdinal(tf.keras.Model):
        def __init__(self, input_dim, num_classes):
            super().__init__()
            self.num_classes = num_classes
            
            # 网络结构
            self.dense_stack = tf.keras.Sequential()
            for size in layer_sizes:
                self.dense_stack.add(tf.keras.layers.Dense(size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
                self.dense_stack.add(tf.keras.layers.Dropout(dropout_rate))
            self.output_layer = tf.keras.layers.Dense(num_classes-1, kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))
            
            # 可训练阈值参数
            self.thresholds = tf.Variable(
                initial_value=tf.sort(tf.linspace(-1.0, 1.0, num_classes-1)),
                trainable=True,
                constraint=MonotonicConstraint(),
                name="ordinal_thresholds"
            )

        def call(self, inputs):
            x = self.dense_stack(inputs)
            return self.output_layer(x)

        def custom_loss(self, y_true, y_pred):
            """序数回归损失函数"""
            y_true = tf.cast(tf.reshape(y_true, (-1, 1)), tf.float32)
            cum_loss = 0.0
            
            for k in range(self.num_classes-1):
                target = tf.cast(y_true > k, tf.float32)
                logit = y_pred[:, k] - self.thresholds[k]
                
                # 添加数值稳定化
                logit = tf.clip_by_value(logit, -10.0, 10.0)
                logit = tf.reshape(logit, (-1, 1))
                
                loss = tf.nn.sigmoid_cross_entropy_with_logits(target, logit)
                cum_loss += tf.reduce_mean(loss)
                
            return cum_loss

    model = DeepOrdinal(input_dim=input_dim, num_classes=num_classes)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        clipvalue=1.0  # 梯度裁剪
    )
    model.compile(
        optimizer=optimizer,
        loss=model.custom_loss,
        metrics=['accuracy']
    )
    return model

# Grid search loop
best_score = float('inf')  # Minimize validation MAE
best_params = None
results = []  # Track all configurations

# 类别权重（根据数据分布调整）
class_weight = {0: 1.0, 1: 2.5, 2: 3.0, 3: 5.0, 4: 10.0}

for layer_sizes, dropout_rate, l2_lambda, lr in itertools.product(layer_configs, dropout_rates, l2_lambdas, learning_rates):
    print(f"Evaluating config: layers={layer_sizes}, dropout={dropout_rate}, l2={l2_lambda}, lr={lr}")
    
    model = build_model(X_subtrain.shape[1], num_classes=5, layer_sizes=layer_sizes, 
                        dropout_rate=dropout_rate, l2_lambda=l2_lambda, learning_rate=lr)
    
    # Train with class weights
    history = model.fit(
        X_subtrain, y_subtrain,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        class_weight=class_weight,
        verbose=0
    )
    
    # Predict on validation and compute MAE
    def ordinal_predict(model, X):
        """将原始输出转换为有序类别"""
        raw_output = model.predict(X, verbose=0)
        thresholds = model.thresholds.numpy()
        cum_probs = tf.sigmoid(raw_output - thresholds)
        binary_probs = tf.cast(cum_probs > 0.5, tf.int32)
        return tf.reduce_sum(binary_probs, axis=1).numpy()

    y_val_pred = ordinal_predict(model, X_val)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    results.append({
        'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr, 'val_mae': val_mae
    })
    
    if val_mae < best_score:
        best_score = val_mae
        best_params = {'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr}

# Save results to CSV for review
pd.DataFrame(results).to_csv('E:/neuro_od/T2D-痴呆结果/grid_search_results.csv', index=False)

print(f"Best configuration: {best_params} with validation MAE: {best_score}")

# Now, use best_params to build and train the final model on full X_train, y_train
model = build_model(
    input_dim=X_train.shape[1],
    num_classes=5,
    layer_sizes=best_params['layers'],
    dropout_rate=best_params['dropout'],
    l2_lambda=best_params['l2'],
    learning_rate=best_params['lr']
)

# ============================ 3.模型训练 =============================
print("\n3. 模型训练中...")

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    class_weight=class_weight,
    verbose=2
)

# =========================== 4.模型评估 =============================
print("\n4. 模型评估...")

y_pred = ordinal_predict(model, X_test)
print(classification_report(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


1. 数据加载与预处理...


C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 

C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [ ]:
# ===================== 5.梯度重要性计算（蛋白质部分）=====================
print("\n5. 计算梯度重要性...")

@tf.function
def compute_gradients(inputs):
    with tf.GradientTape() as tape:
        tape.watch(inputs)  # 显式监视输入
        preds = model(inputs)
    return tape.gradient(preds, inputs)

# 计算所有训练样本的梯度均值
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
gradients = compute_gradients(X_train_tensor)

if gradients is None:
    raise ValueError("梯度计算失败，请检查模型输入输出依赖关系")

# 调整聚合维度（假设输入为二维张量）
abs_gradients = tf.abs(gradients).numpy()  # Convert to NumPy for bootstrapping
protein_abs_gradients = abs_gradients[:, :num_proteins]  # Extract protein part (first num_proteins columns)

# Bootstrapping function for mean importance per protein
def bootstrap_mean(data):
    return np.mean(data, axis=0)  # Mean across samples for each protein

# Perform bootstrapping (95% CI, 1000 resamples)
boot_result = bootstrap((protein_abs_gradients,), bootstrap_mean, n_resamples=1000, random_state=RANDOM_STATE, method='percentile')

# Extract statistics
gradient_importance_mean = boot_result.bootstrap_distribution.mean(axis=-1)  # Mean of bootstraps across resamples; results in shape (67,)
gradient_importance_se = boot_result.standard_error
gradient_importance_ci_low = boot_result.confidence_interval.low
gradient_importance_ci_high = boot_result.confidence_interval.high

# Diagnostic prints to verify lengths (optional; can be removed after confirmation)
print("Length of protein_names:", len(protein_names))
print("Shape of gradient_importance_mean:", gradient_importance_mean.shape)
print("Shape of gradient_importance_se:", gradient_importance_se.shape)
print("Shape of gradient_importance_ci_low:", gradient_importance_ci_low.shape)
print("Shape of gradient_importance_ci_high:", gradient_importance_ci_high.shape)

# Save results to CSV
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': gradient_importance_mean,
    'Gradient Importance SE': gradient_importance_se,
    'Gradient Importance CI Lower (95%)': gradient_importance_ci_low,
    'Gradient Importance CI Upper (95%)': gradient_importance_ci_high
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_dementia_Gradually_Increasing_深度有序回归_with_ci.csv', index=False)

In [ ]:
# 1. Permutation Test
print("\n5.1. Performing Permutation Test...")
n_permutations = 1000  # Adjust for computational feasibility
num_covariates = len(covariates_names)  # e.g., 68 in this run
start_index = num_covariates  # Assume covariates first, proteins next; adjust to 0 if proteins are first

def compute_mean_abs_gradients(inputs):
    grads = compute_gradients(tf.convert_to_tensor(inputs, dtype=tf.float32))
    return np.mean(np.abs(grads.numpy()), axis=0)[start_index:start_index + num_proteins]  # Extract protein part only

# Extract protein gradients consistently
protein_abs_gradients = abs_gradients[:, start_index:start_index + num_proteins]

# Verify data consistency
print("Number of proteins (num_proteins):", num_proteins)
print("Length of protein_names:", len(protein_names))
print("Number of covariates:", num_covariates)
print("Start index for proteins:", start_index)
print("Shape of protein_abs_gradients:", protein_abs_gradients.shape)

# Compute original mean importance for proteins
original_mean = np.mean(protein_abs_gradients, axis=0)
print("Shape of original_mean:", original_mean.shape)

# Generate null distribution by permuting protein columns only
null_distribution = np.zeros((n_permutations, num_proteins))
for i in range(n_permutations):
    permuted_X = X_train.copy()
    for j in range(start_index, start_index + num_proteins):
        np.random.shuffle(permuted_X[:, j])
    null_distribution[i] = compute_mean_abs_gradients(permuted_X)
print("Shape of null_distribution:", null_distribution.shape)

# Compute p-values
p_values_perm = np.array([1 - (percentileofscore(null_distribution[:, j], original_mean[j]) / 100) for j in range(num_proteins)])
print("Length of p_values_perm:", len(p_values_perm))

# Verify lengths before DataFrame creation
if not (len(protein_names) == len(original_mean) == len(p_values_perm)):
    raise ValueError(f"Length mismatch: protein_names ({len(protein_names)}), original_mean ({len(original_mean)}), p_values_perm ({len(p_values_perm)})")

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': original_mean,
    'P-value (Permutation)': p_values_perm
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_permutation_test_Gradually_Increasing.csv', index=False)

In [ ]:
# Simplified; insert after model training
from scipy.stats import norm

# Compute mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Approximate pivotal statistic (e.g., standardized gradient mean)
std_gradients = np.std(protein_abs_gradients, axis=0)
pivotal_stats = original_mean / (std_gradients / np.sqrt(protein_abs_gradients.shape[0]))
p_values_pivotal = 2 * (1 - norm.cdf(np.abs(pivotal_stats)))  # Two-tailed

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Pivotal Statistic': pivotal_stats,
    'P-value (Pivotal)': p_values_pivotal
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_pivotal_test_Gradually_Increasing.csv', index=False)

In [7]:

# ============================= T2D-痴呆结果 Gradually_Increasing =============================

# =============================== 配置区 ===============================
DATA_PATH = "E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Gradually_Decreasing.csv"  # 修改为实际路径
RANDOM_STATE = 42
BATCH_SIZE = 64
EPOCHS = 30
SHAP_SAMPLE_SIZE = 500  # SHAP计算样本量
LEARNING_RATE = 1e-5  # 调低学习率

# ============================= 1.数据准备 =============================
print("\n1. 数据加载与预处理...")

# 加载数据
df = pd.read_csv("E:/neuro_od/T2D-痴呆结果/extracted_data/extracted_Gradually_Decreasing.csv")
protein_names = df.columns[9:63].tolist()
covariates_names = df.columns[2:9].tolist()
num_proteins = len(protein_names)

# 数据拆分
y = df.iloc[:, 1].values.astype(np.int32)
X_protein = df.iloc[:, 9:63].values.astype(np.float32)
X_covariates = df.iloc[:, 2:9].values.astype(np.float32)

# 缺失值处理
imputer = SimpleImputer(strategy='mean')
X_protein = imputer.fit_transform(X_protein)
X_covariates = imputer.fit_transform(X_covariates)

# 标准化
scaler = StandardScaler()
X_protein_scaled = scaler.fit_transform(X_protein)
X_covariates_scaled = scaler.fit_transform(X_covariates)
X_combined = np.hstack([X_protein_scaled, X_covariates_scaled])

# 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=RANDOM_STATE
)

# SMOTE过采样
sm = SMOTE(random_state=RANDOM_STATE)
X_train, y_train = sm.fit_resample(X_train, y_train)

# Further split training data into sub-train and validation for grid search
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=RANDOM_STATE
)

# Define grid search parameters
layer_configs = [[1024, 512, 256, 128], [512, 256, 128], [256, 128]]  # Varying complexity
dropout_rates = [0.2, 0.3, 0.4]
l2_lambdas = [0.001, 0.01, 0.1]
learning_rates = [1e-5, 5e-5, 1e-4]

# Function to build model with variable hyperparameters
class MonotonicConstraint(tf.keras.constraints.Constraint):
    """强制阈值参数单调递增约束"""
    def __call__(self, w):
        return tf.cumsum(tf.nn.elu(w) + 1e-6)

def build_model(input_dim, num_classes, layer_sizes, dropout_rate, l2_lambda, learning_rate):
    class DeepOrdinal(tf.keras.Model):
        def __init__(self, input_dim, num_classes):
            super().__init__()
            self.num_classes = num_classes
            
            # 网络结构
            self.dense_stack = tf.keras.Sequential()
            for size in layer_sizes:
                self.dense_stack.add(tf.keras.layers.Dense(size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
                self.dense_stack.add(tf.keras.layers.Dropout(dropout_rate))
            self.output_layer = tf.keras.layers.Dense(num_classes-1, kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))
            
            # 可训练阈值参数
            self.thresholds = tf.Variable(
                initial_value=tf.sort(tf.linspace(-1.0, 1.0, num_classes-1)),
                trainable=True,
                constraint=MonotonicConstraint(),
                name="ordinal_thresholds"
            )

        def call(self, inputs):
            x = self.dense_stack(inputs)
            return self.output_layer(x)

        def custom_loss(self, y_true, y_pred):
            """序数回归损失函数"""
            y_true = tf.cast(tf.reshape(y_true, (-1, 1)), tf.float32)
            cum_loss = 0.0
            
            for k in range(self.num_classes-1):
                target = tf.cast(y_true > k, tf.float32)
                logit = y_pred[:, k] - self.thresholds[k]
                
                # 添加数值稳定化
                logit = tf.clip_by_value(logit, -10.0, 10.0)
                logit = tf.reshape(logit, (-1, 1))
                
                loss = tf.nn.sigmoid_cross_entropy_with_logits(target, logit)
                cum_loss += tf.reduce_mean(loss)
                
            return cum_loss

    model = DeepOrdinal(input_dim=input_dim, num_classes=num_classes)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        clipvalue=1.0  # 梯度裁剪
    )
    model.compile(
        optimizer=optimizer,
        loss=model.custom_loss,
        metrics=['accuracy']
    )
    return model

# Grid search loop
best_score = float('inf')  # Minimize validation MAE
best_params = None
results = []  # Track all configurations

# 类别权重（根据数据分布调整）
class_weight = {0: 1.0, 1: 2.5, 2: 3.0, 3: 5.0, 4: 10.0}

for layer_sizes, dropout_rate, l2_lambda, lr in itertools.product(layer_configs, dropout_rates, l2_lambdas, learning_rates):
    print(f"Evaluating config: layers={layer_sizes}, dropout={dropout_rate}, l2={l2_lambda}, lr={lr}")
    
    model = build_model(X_subtrain.shape[1], num_classes=5, layer_sizes=layer_sizes, 
                        dropout_rate=dropout_rate, l2_lambda=l2_lambda, learning_rate=lr)
    
    # Train with class weights
    history = model.fit(
        X_subtrain, y_subtrain,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        class_weight=class_weight,
        verbose=0
    )
    
    # Predict on validation and compute MAE
    def ordinal_predict(model, X):
        """将原始输出转换为有序类别"""
        raw_output = model.predict(X, verbose=0)
        thresholds = model.thresholds.numpy()
        cum_probs = tf.sigmoid(raw_output - thresholds)
        binary_probs = tf.cast(cum_probs > 0.5, tf.int32)
        return tf.reduce_sum(binary_probs, axis=1).numpy()

    y_val_pred = ordinal_predict(model, X_val)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    results.append({
        'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr, 'val_mae': val_mae
    })
    
    if val_mae < best_score:
        best_score = val_mae
        best_params = {'layers': layer_sizes, 'dropout': dropout_rate, 'l2': l2_lambda, 'lr': lr}

# Save results to CSV for review
pd.DataFrame(results).to_csv('E:/neuro_od/T2D-痴呆结果/grid_search_results.csv', index=False)

print(f"Best configuration: {best_params} with validation MAE: {best_score}")

# Now, use best_params to build and train the final model on full X_train, y_train
model = build_model(X_train.shape[1], num_classes=5, **best_params)

# ============================ 3.模型训练 =============================
print("\n3. 模型训练中...")

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    class_weight=class_weight,
    verbose=2
)


# =========================== 4.模型评估 =============================
print("\n4. 模型评估...")

y_pred = ordinal_predict(model, X_test)
print(classification_report(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


1. 数据加载与预处理...


C:\Users\HTS\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.01, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.2, l2=0.1, lr=0.0001
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=1e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=5e-05
Evaluating config: layers=[1024, 512, 256, 128], dropout=0.3, l2=0.001, lr=0.0001
Evaluating config: layers=[1024, 

TypeError: build_model() got an unexpected keyword argument 'layers'

In [ ]:
# ===================== 5.梯度重要性计算（蛋白质部分）=====================
print("\n5. 计算梯度重要性...")

@tf.function
def compute_gradients(inputs):
    with tf.GradientTape() as tape:
        tape.watch(inputs)  # 显式监视输入
        preds = model(inputs)
    return tape.gradient(preds, inputs)

# 计算所有训练样本的梯度均值
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
gradients = compute_gradients(X_train_tensor)

if gradients is None:
    raise ValueError("梯度计算失败，请检查模型输入输出依赖关系")

# 调整聚合维度（假设输入为二维张量）
abs_gradients = tf.abs(gradients).numpy()  # Convert to NumPy for bootstrapping
protein_abs_gradients = abs_gradients[:, :num_proteins]  # Extract protein part (first num_proteins columns)

# Bootstrapping function for mean importance per protein
def bootstrap_mean(data):
    return np.mean(data, axis=0)  # Mean across samples for each protein

# Perform bootstrapping (95% CI, 1000 resamples)
boot_result = bootstrap((protein_abs_gradients,), bootstrap_mean, n_resamples=1000, random_state=RANDOM_STATE, method='percentile')

# Extract statistics
gradient_importance_mean = boot_result.bootstrap_distribution.mean(axis=-1)  # Mean of bootstraps across resamples; results in shape (67,)
gradient_importance_se = boot_result.standard_error
gradient_importance_ci_low = boot_result.confidence_interval.low
gradient_importance_ci_high = boot_result.confidence_interval.high

# Diagnostic prints to verify lengths (optional; can be removed after confirmation)
print("Length of protein_names:", len(protein_names))
print("Shape of gradient_importance_mean:", gradient_importance_mean.shape)
print("Shape of gradient_importance_se:", gradient_importance_se.shape)
print("Shape of gradient_importance_ci_low:", gradient_importance_ci_low.shape)
print("Shape of gradient_importance_ci_high:", gradient_importance_ci_high.shape)

# Save results to CSV
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': gradient_importance_mean,
    'Gradient Importance SE': gradient_importance_se,
    'Gradient Importance CI Lower (95%)': gradient_importance_ci_low,
    'Gradient Importance CI Upper (95%)': gradient_importance_ci_high
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_dementia_Gradually_decreasing_深度有序回归_with_ci.csv', index=False)

In [ ]:
from scipy.stats import percentileofscore

# Permutation Test Parameters
n_permutations = 1000  # Adjust for computational feasibility

# Function to compute mean absolute gradients
def compute_mean_abs_gradients(inputs):
    grads = compute_gradients(tf.convert_to_tensor(inputs, dtype=tf.float32))
    return np.mean(np.abs(grads.numpy()), axis=0)[:num_proteins]  # Protein part only

# Original mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Generate null distribution
null_distribution = np.zeros((n_permutations, num_proteins))
for i in range(n_permutations):
    # Permute features (columns) independently for each protein
    permuted_X = X_train.copy()
    for j in range(num_proteins):
        np.random.shuffle(permuted_X[:, j])
    null_distribution[i] = compute_mean_abs_gradients(permuted_X)

# Compute p-values (one-tailed: proportion of null means >= original)
p_values_perm = np.array([1 - (percentileofscore(null_distribution[:, j], original_mean[j]) / 100) for j in range(num_proteins)])

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Gradient Importance Mean': original_mean,
    'P-value (Permutation)': p_values_perm
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_permutation_test_Gradually_Decreasing.csv', index=False)

In [ ]:
# Simplified; insert after model training
from scipy.stats import norm

# Compute mean importance
original_mean = np.mean(protein_abs_gradients, axis=0)

# Approximate pivotal statistic (e.g., standardized gradient mean)
std_gradients = np.std(protein_abs_gradients, axis=0)
pivotal_stats = original_mean / (std_gradients / np.sqrt(protein_abs_gradients.shape[0]))
p_values_pivotal = 2 * (1 - norm.cdf(np.abs(pivotal_stats)))  # Two-tailed

# Save results
pd.DataFrame({
    'Protein': protein_names,
    'Pivotal Statistic': pivotal_stats,
    'P-value (Pivotal)': p_values_pivotal
}).to_csv('E:/neuro_od/T2D-痴呆结果/gradient_importance_pivotal_test_Gradually_Decreasing.csv', index=False)